# FFF Workshop

## A5: Algorithmic fragment merging

### Outline

- Identifying merging opportunities
- Small scale Fragmenstein merge via API
- Small scale Knitwork
- Point to larger scale stuff in day 2
- Squonk jobs

## Identifying merging opportunities

For fragment progression it is often desirable to restrict the observations considered for merging. When working with Fragalysis data, tags and Canonical Sites assigned by XCA can be useful starting points

In [ ]:
# setup the animal
import hippo
animal = hippo.HIPPO(
    "A71EV2A_demo",
    "../data/A71EV2A.sqlite",
)

For the example target A71EV2A it makes most sense to explore merges of fragments in the active site:

In [ ]:
active_site_fragments = animal.poses(tag="[Other] Active site fragment")
active_site_fragments

In [ ]:
active_site_hits = animal.poses(tag="hits").get_by_subsite(id=1)
active_site_hits

You may also want to explore merges of other/larger ligands in the active site:

One way to anticipate pairs of molecules that are suitable for merging is to look at their interactions. An ideal merge will combine molecules that share some but not all interactions with each other. Consider the following pair:

In [ ]:
animal.poses["A0207a", "A0237a"].draw()

The overlapping ring might make it suitable for merging. Looking at the interactions:

In [ ]:
animal.poses["A0207a"].interactions.summary()
animal.poses["A0237a"].interactions.summary()

There are 2 shared interactions, which could be exploited during merging.

In HIPPO a `PoseSet` method exists to count these opportunities using interactions in the dataset:

In [ ]:
animal.poses["A0207a", "A0237a"].get_interaction_overlaps()

This tells us that `A0207a` and `A0237a` might be worth trying to merge.

For a larger set you can count all the interaction overlaps as well:

In [ ]:
active_site_fragments.get_interaction_overlaps()

You can also get the pairs as a list of `PoseSet` objects:

In [ ]:
pairs = active_site_fragments.get_interaction_overlaps(return_pairs=True)

for pair in pairs:
    print(pair.names)
    break

## Submitting Jobs to Squonk

Squonk is the job and data management service used by Fragalysis to submit jobs, such as this notebook app!

Squonk provides a Data Manager (DM) user interace: https://data-manager-ui.xchem.diamond.ac.uk/data-manager-ui

The DM can be used to submit and monitor jobs, as well as see outputs via it's interface. 

In this notebook though, we'll cover how to programmatically do the same. Jobs can be submitted using a Fragalysis token, but to monitor them, a squonk token is needed.

### Getting a Squonk access token

1. Go to the Squonk Data Manager API (Swagger) page: https://data-manager.xchem.diamond.ac.uk/data-manager-api/api/
2. Click `Authorize`
3. Enter `data-manager-api-production` as the `client-id`
4. Click Authorize
5. Log in via Diamond CAS
6. Scroll down to https://data-manager.xchem.diamond.ac.uk/data-manager-api/api/#/user/app.api_user.get_api_token
7. Click `Try it out` and then `Execute`
8. Copy the token value from the `Response body` and paste it in the cell below:

In [ ]:
squonk_token = "eyJhbGciO ... iqTeA"

# save it to environment variable for easier reuse across notebooks
from os import environ
environ["SQUONK_TOKEN"] = squonk_token

Also do the same for your Fragalysis token:

In [ ]:
token = "kjqi8v9c ... 6gud2x"

# save it to environment variable for easier reuse across notebooks
from os import environ
environ["FRAGALYSIS_TOKEN"] = token

## Fragmenstein Merging

Fragmenstein can be used to generate merges of multiple molecules, and minimise their conformations relative to the reference/inspiration molecules.

A Fragmenstein merging job is available in Fragalysis/Squonk and can be requested with the fragalysis Python API. 

N.B. This does require all referenced observations and the protein conformation to already be present in Fragalysis.

See also: [Fragmenstein paper](https://doi.org/10.1186/s13321-025-00946-0)

In [ ]:
from fragalysis.requests import fragmenstein_combine, monitor_jobs, get_file

In [ ]:
fragmenstein_combine(
    observations=["A0207a", "A0237a"], 
    protein="A0207a",
    target_name="A71EV2A", 
    tas="lb32627-66", 
    stack="staging", 
    token=environ["FRAGALYSIS_TOKEN"],
)

The returned link can be used to monitor the job and retrieve the outputs via the Squonk Data Manager UI.

All jobs submitted to Squonk can also be viewed as a live table:

In [ ]:
monitor_jobs(
    stack="staging",
    token=squonk_token,
)

To get the output file from the job use:

In [ ]:
get_file(
    instance="instance-8ecb4d8f-71e1-43f6-b975-7c7f5dba0d7d",
    path="fragalysis-files/hnge/merged.sdf",
    destination="merged.sdf",
    token=environ["SQUONK_TOKEN"],
    stack="staging",
)

## Fragment Knitwork merging

An alternative method to fragment merging using a graph database called *Fragment Knitwork* can also be accessed via Squonk.

See also: [Fragment Knitwork paper](https://doi.org/10.1021/acs.jcim.3c00276)

In [ ]:
from fragalysis.requests import knitwork
from fragalysis.requests.squonk import list_files, monitor_jobs

In [ ]:
knitwork(
    observations=["A0237a", "A0207a"],
    target_name="A71EV2A",
    tas="lb32627-66",
    stack="staging",
    token=environ["FRAGALYSIS_TOKEN"],
)

Similarly, monitor the job and fetch it's results:

In [ ]:
monitor_jobs(
    stack="staging",
    token=environ["SQUONK_TOKEN"],
)

In [ ]:
list_files(
    instance="instance-b4232c91-f5a5-418b-853a-943b7624ac9d",
    root="fragalysis-files/gtap",
    token=environ["SQUONK_TOKEN"],
)

## Loading results back into HIPPO